In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Model
from itertools import groupby
import os
import pandas as pd

Using TensorFlow backend.
/Users/prasadv/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/prasadv/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/prasadv/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/prasadv/miniconda3/lib/python3.6/site-pa

In [2]:
model = ResNet50(weights='imagenet')

In [3]:
vecModel = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)

In [22]:
def getVectors(info,videoName):
    img = image.load_img('testingImages2/'+videoName+'/'+info[0], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return vecModel.predict(x)[0]

token2Num = {'Burpees':1,'JumpingJacks':2,'MountainClimber':3,'Plank':4,'PushUps':5,'Squat':6,'Other':0}
num2Token = {1:'Burpees',2:'JumpingJacks',3:'MountainClimber',4:'Plank',5:'PushUps',6:'Squat',0:'Other'}
tokenArray=['Other','Burpees','JumpingJacks','MountainClimber','Plank','PushUps','Squat']
def getNum(x): return token2Num.get(x, 0)
def getToken(x): return num2Token.get(x, 'Other')

In [5]:
# session_id, label, seconds, i
def generateEmbeddingsForVideo(videoName):
    files = list(map(lambda x:[x,x.split('_')],list(filter(lambda x:'.png' in x,os.listdir('testingImages2/'+videoName+'/')))))
    files.sort(key=lambda x:x[1][0])

    out = []
    for key, group in groupby(files, lambda x: x[1][0]):
        group = list(group)
        for key2, group2 in groupby(group, lambda x: x[1][3]):
            group2=list(group2)
            if(len(group)>=5):
                vecs=[]
                for info in group:
                    vecs.append(getVectors(info,videoName))
                for i in range(5,len(vecs)+1):
                    out.append([getNum(info[1][1]), np.concatenate(vecs[i-5:i])])
    df = pd.DataFrame(out,columns =['label', 'features'])
    df.to_pickle('testingEmbeddings2/'+videoName)
    print('Saved embeddings for video '+videoName)

for video in ['GroupWorkoutSideView','GroupWorkoutVideo','SingleWorkoutSideView','SingleWorkoutVideo']:
    generateEmbeddingsForVideo(video)

Saved embeddings for video GroupWorkoutSideView
Saved embeddings for video GroupWorkoutVideo
Saved embeddings for video SingleWorkoutSideView
Saved embeddings for video SingleWorkoutVideo


In [6]:
# df = pd.concat(list(map(lambda x:pd.read_pickle('testingEmbeddings2/'+x),['GroupWorkoutSideView','GroupWorkoutVideo','SingleWorkoutSideView','SingleWorkoutVideo'])))

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [8]:
# df = pd.read_pickle('trainingEmbeddings/DIY015')

In [9]:
# dataset = df.values
# X = np.stack(dataset[:,1], axis=0)
# Y = dataset[:,0]
# dummy_y = np_utils.to_categorical(Y)

In [10]:
from keras.models import load_model
loaded_model=load_model('modelV1')


In [50]:
metrics= []
for i in ['GroupWorkoutSideView','GroupWorkoutVideo','SingleWorkoutSideView','SingleWorkoutVideo']:
    df=pd.read_pickle('testingEmbeddings2/'+i)
    dataset = df.values
    X = np.stack(dataset[:,1], axis=0)
    Y = dataset[:,0]
    dummy_y = np_utils.to_categorical(Y)
    predictions = loaded_model.predict_classes(X)
    print(i)
    for j in list(map(lambda x:[getToken(x[0]),getToken(x[1])],list(zip(predictions,Y)))):
        print(j)
    t = list(map(lambda x:x[0]==x[1],list(zip(predictions,Y))))
    metrics.append('{} \t True : {}\t False : {}\t Accuracy : {}'.format(i,sum(t),abs(sum(t)-len(t)),(sum(t)/len(t)*100)))
    print(" ")

GroupWorkoutSideView
['MountainClimber', 'JumpingJacks']
['MountainClimber', 'JumpingJacks']
['MountainClimber', 'JumpingJacks']
['Squat', 'JumpingJacks']
['MountainClimber', 'JumpingJacks']
['MountainClimber', 'Plank']
['MountainClimber', 'Plank']
['MountainClimber', 'Plank']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['MountainClimber', 'PushUps']
['MountainClimber', 'PushUps']
['Plank', 'PushUps']
['Plank', 'PushUps']
['MountainClimber', 'Burpees']
['MountainClimber', 'Burpees']
['MountainClimber', 'Burpees']
['MountainClimber', 'Burpees']
['MountainClimber', 'MountainClimber']
['MountainClimber', 'MountainClimber']
['MountainClimber', 'MountainClimber']
['MountainClimber', 'MountainClimber']
 
GroupWorkoutVideo
['Squat', 'JumpingJacks']
['Squat', 'Plank']
['Squat', 'Plank']
['Squat', 'Plank']
['MountainClimber', 'Plank']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['Squat', 'Squat']
['MountainClimbe

In [51]:
for i in metrics:
    print(i)

GroupWorkoutSideView 	 True : 9	 False : 16	 Accuracy : 36.0
GroupWorkoutVideo 	 True : 8	 False : 16	 Accuracy : 33.33333333333333
SingleWorkoutSideView 	 True : 2	 False : 21	 Accuracy : 8.695652173913043
SingleWorkoutVideo 	 True : 0	 False : 18	 Accuracy : 0.0
